# Optical Example notebook

This notebook will outline basic usage of DefDAP, including loading an image and EBSD map, linking them with homologous points and producing maps

## Load in packages

DefDAP is split into modules for processing EBSD (`defdap.ebsd`) and image (`defdap.optical`) data. There are also modules for manpulating orientations (`defdap.quat`) and creating custom figures (`defdap.plotting`) which is introduced later. We also import some of the usual suspects of the python scientific stack: `numpy` and `matplotlib`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import defdap.optical as optical
import defdap.ebsd as ebsd
import defdap.plotting as plotting
from defdap.quat import Quat

# try tk, qt, osx (if using mac) or notebook for interactive plots. If none work, use inline
%matplotlib tk

## Load in a optical/SEM image

In [2]:
optical_filepath = "../tests/data/testOptical.png"
optical_map = optical.Map(file_name = optical_filepath)

(dimensions: 1170 x 935 pixels)


### Set the scale of the map
This is defined as the pixel size in the images, measured in microns per pixel.

In [3]:
optical_map.set_scale(2.105) # um/pixel

### Plot the map with a scale bar

In [ ]:
optical_map.plot_map('image', vmin=0, vmax=1, plot_scale_bar=True)

### Crop the map
Optical maps can contain spurious data at the edges which should be removed before performing any analysis. The crop is defined by the number of points to remove from each edge of the map, where `xMin`, `xMax`, `yMin` and `yMax` are the left, right, top and bottom edges respectively. Note that the test data doesn not require cropping as it is a subset of a larger dataset.

In [4]:
optical_map.set_crop(left=10, right=10, top=10, bottom=10)

## Load in an EBSD map
Currently, OxfordBinary (a .crc and .cpr file pair), OxfordText (.ctf file), EdaxAng (.ang file) or PythonDict (Python dictionary) filetypes are supported. The crystal structure and slip systems are automatically loaded for each phase in the map. The orientation in the EBSD are converted to a quaternion representation so calculations can be applied later.

In [5]:
ebsd_map = ebsd.Map("../tests/data/f-5-test-region")

Loaded EBSD data (dimensions: 852 x 734 pixels, step size: 3.0 um)


A list of detected phases and crystal structures can be printed

In [ ]:
for i, phase in enumerate(ebsd_map.phases):
    print(i+1)
    print(phase)

A list of the slip planes, colours and slip directions can be printed

In [ ]:
ebsd_map.phases[0].print_slip_systems()

### Plot the EBSD map
Using an Euler colour mapping or inverse pole figure colouring with the sample reference direction passed as a vector.

In [ ]:
ebsd_map.plot_map('euler_angle', 'all_euler', plot_scale_bar=True)

In [ ]:
ebsd_map.plot_map('orientation', 'IPF_x', plot_scale_bar=True)

A KAM map can also be plotted as follows

In [ ]:
ebsd_map.plot_map('KAM', vmin=0, vmax=2*np.pi/180)

### Detect grains in the EBSD
This is done in two stages: first bounaries are detected in the map as any point with a misorientation to a neighbouring point greater than a critical value (`boundDef` in degrees). A flood fill type algorithm is then applied to segment the map into grains, with any grains containining fewer than a critical number of pixels removed (`minGrainSize` in pixels). The data e.g. orientations associated with each grain are then stored (referenced strictly, the data isn't stored twice) in a grain object and a list of the grains is stored in the EBSD map (named `grainList`). This allows analysis routines to be applied to each grain in a map in turn.

In [6]:
ebsd_map.data.generate('grain_boundaries', misori_tol=8)
ebsd_map.data.generate('grains', min_grain_size=200)

Finished building quaternion array (0:00:00) 
Finished finding grain boundaries (0:00:01) 
Finished finding grains (0:00:00) 


The Schmid factors for each grain can be calculated and plotted. The `slipSystems` argument can be specified, to only calculate the Schmid factor for certain planes, otherwise the maximum for all slip systems is calculated.

In [7]:
ebsd_map.calc_average_grain_schmid_factors(load_vector=np.array([1,0,0]), slip_systems=None)

Finished calculating grain average Schmid factors (0:00:01) 


In [ ]:
ebsd_map.plot_average_grain_schmid_factors_map()

### Single grain analysis
The `locate_grain` method allows interactive selection of a grain of intereset to apply any analysis to. Clicking on grains in the map will highlight the grain and print out the grain ID (position in the grain list) of the grain.

In [ ]:
ebsd_map.locate_grain()

A built-in example is to calculate the average orientation of the grain and plot this orientation in a IPF

In [ ]:
grain_id = 48
grain = ebsd_map[grain_id]
grain.calc_average_ori()  # stored as a quaternion named grain.refOri
print(grain.ref_ori)
grain.plot_ref_ori(direction=[0, 0, 1])

The spread of orientations in a given grain can also be plotted on an IPF

In [ ]:
plot = grain.plot_ori_spread(direction=np.array([0, 0, 1]), c='b', s=1, alpha=0.2)
grain.plot_ref_ori(direction=[0, 0, 1], c='k', plot=plot)

The unit cell for the average grain orientation can also be ploted

In [ ]:
grain.plot_unit_cell()

Printing a list of the slip plane indices, angle of slip plane intersection with the screen (defined as counter-clockwise from upwards), colour defined for the slip plane and also the slip directions and corresponding Schmid factors, is also built in

In [ ]:
grain.print_slip_traces()

A second built-in example is to calcuate the grain misorientation, specifically the grain reference orientation deviation (GROD). This shows another feature of the `locate_grain` method, which stores the last selected grain in a variable called `sel_grain` in the EBSD map.

In [ ]:
if ebsd_map.sel_grain == None: 
    ebsd_map.sel_grain = ebsd_map[57]
    
ebsd_map.sel_grain.build_mis_ori_list()
ebsd_map.sel_grain.plot_mis_ori(plot_scale_bar=True, vmin=0, vmax=5)

### Multi grain analysis
Once an analysis routine has been prototyped for a single grain it can be applied to all the grains in a map using a loop over the grains and any results added to a list for use later. Of couse you could also apply to a smaller subset of grains as well.

In [ ]:
grain_av_oris = []
for grain in ebsd_map:
    grain.calc_average_ori()
    grain_av_oris.append(grain.ref_ori)

# Plot all the grain orientations in the map
Quat.plot_ipf(grain_av_oris, [0, 0, 1], ebsd_map.crystal_sym, marker='o', s=10)
plt.tight_layout()

Some common grain analysis routines are built into the EBSD map object, including:

In [ ]:
ebsd_map.calc_grain_av_oris()

In [ ]:
ebsd_map.calc_grain_mis_ori()
ebsd_map.plot_mis_ori_map(vmin=0, vmax=5, plot_gbs=True, plot_scale_bar=True)

There are also methods for plotting GND density, phases and boundaries. All of the plotting functions in DefDAP use the same parameters to modify the plot, examples seen so far are `plot_gbs`, `plotScaleBar`, `vmin`, `vmax`.

## Linking the optical and EBSD maps
### Define homologous points
To register the two datasets, homologous points (points at the same material location) within each map are used to estimate a transformation between the two frames the data are defined in. The homologous points are selected manually using an interactive tool within DefDAP. To select homologous call the method `setHomogPoint` on each of the data maps, which will open a plot window with a button labelled 'save point' in the bottom right. You select a point by right clicking on the map, adjust the position with the arrow  and accept the point by with the save point button. Then select the same location in the other map.

In [ ]:
optical_map.set_homog_point()

In [ ]:
ebsd_map.set_homog_point()

The points are stored as a list of tuples `(x, y)` in each of the maps. This means the points can be set from previous values.

In [ ]:
optical_map.frame.homog_points

In [ ]:
ebsd_map.frame.homog_points

Here are some example homologous points for this data, after setting these by running the cells below you can view the locations in the maps by running the `setHomogPoint` methods (above) again

In [8]:
optical_map.frame.homog_points = [
    (467, 653), 
    (806, 239), 
    (180, 94)
]

In [9]:
ebsd_map.frame.homog_points = [
    (314, 533), 
    (543, 242), 
    (97, 143)
]

### Link the maps
Finally the two data maps are linked. The type of transform between the two frames can be affine, projective, polynomial.

In [10]:
optical_map.link_ebsd_map(ebsd_map, transform_type="affine")
# optical_map.link_ebsd_map(ebsd_map, transform_type="projective")
# optical_map.link_ebsd_map(ebsd_map, transform_type="polynomial", order=2)

### Show the transformation

In [ ]:
from skimage import transform as tf

data = np.zeros((2000, 2000), dtype=float)
data[500:1200, 500:1200] = 1.
transform = optical_map.experiment.get_frame_transform(optical_map.frame, ebsd_map.frame)
dataWarped = tf.warp(data, transform)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,4))
ax1.set_title('Reference')
ax1.imshow(data)
ax2.set_title('Transformed')
ax2.imshow(dataWarped)

### Segment into grains
The optical map can now be segmented into grains using the grain boundaries detected in the EBSD map. Analysis rountines can then be applied to individual grain, as with the EBSD grains. The grain finding process will also attempt to link the grains between the EBSD and optical maps and each grain in the optical image has a reference (`ebsdGrain`) to the corrosponding grain in the EBSD map.

In [11]:
optical_map.data.generate('grains', algorithm='warp', min_grain_size=10)

Finished finding grains (0:00:00) 


In [ ]:
optical_map.plot_map('image', vmin=0, vmax=1, plot_scale_bar=True, plot_gbs='line')

Now, a grain can also be selected interactively in the optical map, in the same way a grain can be selected from an EBSD map. If `displaySelected` is set to true, then a pop-out window shows the map segmented for the grain

In [ ]:
optical_map.locate_grain(display_grain=True)

### Grain inspector
This is an interactive tool used to perform slip trace analysis

In [12]:
optical_map.grain_inspector()